In [ ]:
import torch
from util.metric import eval_disparity
import numpy as np

In [ ]:
def count_parameters(model):
    print(sum(p.numel() for p in model.parameters()))
    return None

In [ ]:
from model.depth_anything_v2.dpt_align_lora import DepthAnythingV2_lora
student_model = DepthAnythingV2_lora(encoder="vitl")


In [ ]:
count_parameters(student_model.pretrained)

In [ ]:
d1 = "/data/coding/code/da2-prompt-tuning/vis_feature_maps/align_eventscape_event_1036_1_da2/npy/05_084_0124_image.npy"
d2 = "/data/coding/code/da2-prompt-tuning/vis_feature_maps/align_eventscape_event_1036_1/npy/05_084_0124_image.npy"

pred = torch.tensor(np.load(d2))
target = torch.tensor(np.load(d1))
mask = torch.ones_like(pred, dtype=bool)
pred = pred[mask]
target = target[mask]
pred = 1.0 / (pred + 1e-8)
target = 1.0 / (target + 1e-8)
# print(pred.shape, target.shape)
# thresh = torch.max((target / pred), (pred / target))
# # print(len(thresh))
# d1 = torch.sum(thresh < 1.25).float()
# print(d1)
# # print(len(d1))
print(eval_disparity(pred, target))

In [ ]:
p = r"/home/sph/event/da2-prompt-tuning/exp/epde_nl_mvsec_2_decoder_20250116_222954/latest.pth"
state_dict = torch.load(p)
# print(state_dict.keys())

# previous_best = state_dict["previous_best"]
# print(previous_best)
state_dict = state_dict['model']
print("Model's state_dict keys:")
for key in state_dict.keys():
    # print(state_dict[key].dtype)
    print(key)

#### Test corruption

In [ ]:
def print_para(module):
    for name, param in module.named_parameters():
        if param.requires_grad:
            print(f"Module: {name}, Trainable Parameters: {param.numel()}")

In [ ]:
import loralib as lora
from torch import nn

embed_dim = 1024
linear = nn.Linear(embed_dim, embed_dim)
linear_lora = lora.Linear(embed_dim, embed_dim, r=8)

print_para(linear)
print("*****************************")
print_para(linear_lora)

In [ ]:
in_chans = 3
patch_HW = (14,14)
proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_HW, stride=patch_HW)
proj_lora = lora.Conv2d(in_chans, embed_dim, kernel_size=patch_HW[0], stride=patch_HW[0], r=8)

print_para(proj)
print("*****************************")
print_para(proj_lora)

In [ ]:
linear = nn.Linear(embed_dim, embed_dim*3)
linear_lora = lora.Linear(embed_dim, embed_dim*3, r=8)
print_para(linear)
print("*****************************")
print_para(linear_lora)

In [ ]:
linear = nn.Linear(embed_dim, embed_dim*3)

# Alternatively, use lora.MergedLinear (recommended)
qkv_proj = lora.MergedLinear(embed_dim, 3*embed_dim, r=8, enable_lora=[True, True, True], bias=True)

print_para(linear)
print("*****************************")
print_para(qkv_proj)

In [ ]:
m1 = lora.Linear(10, 10)
m2 = lora.MergedLinear(10, 20, enable_lora=[True, True])
m3 = lora.Conv2d(10, 10, 1)

if isinstance(m1, nn.Linear):
    print("m1")
if isinstance(m2, nn.Linear):
    print("m2")
if isinstance(m3, nn.Conv2d):
    print("m3")

In [ ]:
import torch

p = '/data/coding/code/da2-prompt-tuning/da2_metric_depth/exp/eventscape_voxel_nl_disp_da2vitl_20250107_000222/abs_rel-0.15593145787715912-0.pth'
p = '/data/coding/code/da2-prompt-tuning/da2_metric_depth/exp/eventscape_voxel_nl_disp_da2vitl_20250107_000222/latest.pth'
p = '/data/coding/upload-data/checkpoints/depth_anything_v2_vitl.pth'
state_dict = torch.load(p, map_location="cpu")
print(state_dict.keys())

In [ ]:
state_dict = state_dict['model']
print(state_dict.keys())

In [ ]:
import os
def gen_split(scene, data_dir, output_dir="./dataset/splits/mvsec/"):
    depths_dir = os.path.join(data_dir, "depths")
    images_dir = os.path.join(data_dir, "images")
    voxels_dir = os.path.join(data_dir, "voxels")
    
    depths = os.listdir(depths_dir)
    images = os.listdir(images_dir)
    voxels = os.listdir(voxels_dir)
    
    depths.sort()
    images.sort()
    voxels.sort()
    
    assert len(depths) == len(images) and len(depths) == len(voxels)
    
    depths = [os.path.join(depths_dir, file_name) for file_name in depths]
    images = [os.path.join(images_dir, file_name) for file_name in images]
    voxels = [os.path.join(voxels_dir, file_name) for file_name in voxels]
    
    lines = []
    for i in range(len(depths)):
        line = images[i] + ' ' + depths[i] + ' ' + voxels[i] + '\n'
        lines.append(line)
    
    os.makedirs(output_dir, exist_ok=True)
    if scene == "outdoor_day2": # Train DataSet
        with open(output_dir + "train.txt", 'w') as f:
            f.writelines(lines)
    else:
        # Test & Val DataSet
        # TODO: 
        with open(output_dir + scene + ".txt", 'a') as f:
            f.writelines(lines)
        
        # lines = random.sample(lines, 100)
        # with open(output_dir + scene + "_val.txt", 'a') as f:
        #     f.writelines(lines)
    print(f"Saved split file to {output_dir}")

In [ ]:
process_list = [
        # ("outdoor_day1_data.hdf5", "outdoor_day1_gt.hdf5"),
        # ("outdoor_day2_data.hdf5", "outdoor_day2_gt.hdf5"),
        # ("outdoor_night1_data.hdf5", "outdoor_night1_gt.hdf5"),
        ("outdoor_night2_data.hdf5", "outdoor_night2_gt.hdf5"),
        ("outdoor_night3_data.hdf5", "outdoor_night3_gt.hdf5"),
]

root = '/data/coding/upload-data/data/mvsec'
for pair in process_list:
    # outdoor_day1, outdoor_day2 ...
    scene = pair[0].split('_')[0] + '_' + pair[0].split('_')[1]

    output_dir = os.path.join(root, scene)
    gen_split(scene, output_dir)

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from model.epde.utils import token2feature, feature2token

In [ ]:
def count_parameters(model):
    print(sum(p.numel() for p in model.parameters() if p.requires_grad))
    return None

In [ ]:
class FeatureFusionWeight(nn.Module):
    def __init__(self, in_channels, kernel_size=3):
        super().__init__()
        weight_layers = [
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=in_channels,
                kernel_size=kernel_size,
            )
        ]
        weight_layers.append(nn.GELU())
        weight_layers.append(
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=2,
                kernel_size=kernel_size,
            )
        )
        self.get_weight = nn.Sequential(*weight_layers)
    
    def forward(self, joint_feas):
        # The shape of x1 and x2 is [B, C, H, W]
        weight = self.get_weight(joint_feas)
        weight = F.softmax(weight, dim=-1)
        return weight

In [ ]:
class CrossAttentionBlock(nn.Module):
    def __init__(self, dim, num_heads, dropout=0.1):
        super(CrossAttentionBlock, self).__init__()

        self.dim = dim
        self.num_heads = num_heads
        self.head_dim = dim // num_heads
        assert self.head_dim * num_heads == dim, "dim must be divisible by num_heads"
        self.scale = self.head_dim**-0.5

        self.query = nn.Linear(dim, dim)
        self.key = nn.Linear(dim, dim)
        self.value = nn.Linear(dim, dim)
        self.out = nn.Linear(dim, dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value):
        B, L, C = query.shape

        # (B, num_heads, L, head_dim)
        Q = self.query(query).view(B, L, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.query(key).view(B, L, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.value(value).view(B, L, self.num_heads, self.head_dim).transpose(1, 2)

        # Scaled dot-product attention (B, num_heads, L, L)
        scores = torch.matmul(Q, K.transpose(-2, -1)) * self.scale

        attn_weight = F.softmax(scores, dim=-1)
        attn_weight = self.dropout(attn_weight)

        # (B, num_heads, L, head_dim)
        context = torch.matmul(attn_weight, V)
        # (B, L, dim)
        context = context.transpose(1, 2).contiguous().view(B, L, self.dim)

        return self.out(context)


class FeatureInteraction(nn.Module):
    def __init__(self, dim, num_heads, dropout=0.1):
        super(FeatureInteraction, self).__init__()
        self.cross_attention_1 = CrossAttentionBlock(dim, num_heads, dropout)
        self.cross_attention_2 = CrossAttentionBlock(dim, num_heads, dropout)
    
    def forward(self, x1, x2):
        inter_x1 = self.cross_attention_1(query=x1, key=x2, value=x2)
        inter_x2 = self.cross_attention_2(query=x2, key=x1, value=x1)
        
        return inter_x1, inter_x2

In [ ]:
class SpatialWeights(nn.Module):
    def __init__(self, dim, reduction=1):
        super(SpatialWeights, self).__init__()
        self.dim = dim
        self.mlp = nn.Sequential(
            nn.Conv2d(self.dim * 2, self.dim // reduction, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(self.dim // reduction, 2, kernel_size=1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        # B, _, H, W = x1.shape
        # x = torch.cat((x1, x2), dim=1)  # B 2C H W
        return self.mlp(x)
    # def forward(self, x1, x2):
    #     B, _, H, W = x1.shape
    #     x = torch.cat((x1, x2), dim=1)  # B 2C H W
    #     return self.mlp(x)

In [ ]:
class FeatureFusionModule(nn.Module):
    def __init__(self, dim, num_heads=None, norm_layer=nn.BatchNorm2d):
        super().__init__()
        self.upsample = nn.PixelShuffle(upscale_factor=2)
        self.interact = FeatureInteraction(dim=dim//4, num_heads=num_heads)
        self.downsample_1 = nn.Conv2d(
            in_channels=dim // 4, out_channels=dim, kernel_size=1, stride=2
        )
        self.downsample_2 = nn.Conv2d(
            in_channels=dim // 4, out_channels=dim, kernel_size=1, stride=2
        )
        self.fuse_weight = FeatureFusionWeight(
            in_channels=dim // 2, kernel_size=1
        )
    
    def fuse_feature_maps(self, weight_map, feature_map1, feature_map2):
        # Split the weight map into two components, [B, 1, H, W]
        weight1 = weight_map[:, 0:1, :, :]
        weight2 = weight_map[:, 1:2, :, :]
        
        # Apply weights to the feature maps, [B, C, H, W] * [B, 1, H, W]
        weighted_feature1 = feature_map1 * weight1
        weighted_feature2 = feature_map2 * weight2

        # Compute the fused feature map, [B, C, H, W]        
        return weighted_feature1 + weighted_feature2

    def forward(self, x1, x2, patch_grid_size):
        assert x1.shape == x2.shape, "The shape of x1 and x2 does not match."
        # [B, L, C] --> [B, C, H, W]
        x1 = token2feature(x1, patch_grid_size)
        x2 = token2feature(x2, patch_grid_size)

        # [B, C, H, W] --> [B, C/4, H*2, W*2]
        H, W = patch_grid_size
        upsample_patch_grid_size = (H * 2, W * 2)
        x1 = self.upsample(x1)
        x2 = self.upsample(x2)
        
        # [B, C/4, H*2, W*2] --> [B, L*4, C/4]
        x1 = feature2token(x1)
        x2 = feature2token(x2)
        x1, x2 = self.interact(x1, x2)
        
        # [B, L*4, C/4] --> [B, C/4, H*2, W*2]
        x1 = token2feature(x1, upsample_patch_grid_size)
        x2 = token2feature(x2, upsample_patch_grid_size)
        
        # [B, C/4, H*2, W*2] --> [B, C, H, W]
        x1_down = self.downsample_1(x1)
        x2_down = self.downsample_2(x2)
        
        # [B, C/4, H*2, W*2] * 2 --> [B, C/2, H*2, W*2]
        joint_feas = torch.cat((x1, x2), dim=1)
        weight = self.fuse_weight(joint_feas)
        fused_feature_map = self.fuse_feature_maps(weight, x1_down, x2_down)
        return feature2token(fused_feature_map)

In [ ]:
from model.epde.prompt_module import FeatureInteraction
from model.epde.prompt_module import FeatureFusionWeight
class FeatureFusionModule(nn.Module):
    def __init__(self, dim, num_heads=None, reduction=1, norm_layer=nn.LayerNorm):
        super().__init__()
        self.channel_down_proj1 = nn.Linear(dim, dim // reduction)
        self.channel_down_proj2 = nn.Linear(dim, dim // reduction)
        self.interact = FeatureInteraction(dim=dim // reduction, num_heads=num_heads)
        self.channel_up_proj1 = nn.Linear(dim // reduction, dim)
        self.channel_up_proj2 = nn.Linear(dim // reduction, dim)
        self.norm1 = norm_layer(dim)
        self.norm2 = norm_layer(dim)
        self.fuse_weight = SpatialWeights(dim=dim, reduction=reduction)
    
    def fuse_feature_maps(self, weight_map, feature_map1, feature_map2):
        # Split the weight map into two components, [B, 1, H, W]
        weight1 = weight_map[:, 0:1, :, :]
        weight2 = weight_map[:, 1:2, :, :]
        
        # Apply weights to the feature maps, [B, C, H, W] * [B, 1, H, W]
        weighted_feature1 = feature_map1 * weight1
        weighted_feature2 = feature_map2 * weight2

        # Compute the fused feature map, [B, C, H, W]        
        return weighted_feature1 + weighted_feature2

    def forward(self, x1, x2, patch_grid_size):
        assert x1.shape == x2.shape, "The shape of x1 and x2 does not match."
        
        y1 = self.channel_down_proj1(x1)
        y2 = self.channel_down_proj2(x2)
        
        y1, y2 = self.interact(y1, y2)
        
        y1 = self.channel_up_proj1(y1)
        y2 = self.channel_up_proj2(y2)
        
        print(y1.shape, x1.shape)
        y1 = self.norm1(y1 + x1)
        y2 = self.norm2(y2 + x2)
        
        y1 = token2feature(y1, patch_grid_size)
        y2 = token2feature(y2, patch_grid_size)
        
        joint_feas = torch.cat((y1, y2), dim=1)
        weight = self.fuse_weight(joint_feas)
        fused_feature_map = self.fuse_feature_maps(weight, y1, y2)
        return feature2token(fused_feature_map)

In [ ]:
ffm = FeatureFusionModule(dim=1024, num_heads=16, reduction=8)
count_parameters(ffm.interact)
count_parameters(ffm.fuse_weight)
count_parameters(ffm.channel_down_proj1)
count_parameters(ffm.channel_up_proj1)
count_parameters(ffm)

In [ ]:
B = 4  # Batch size
C = 1024  # Channels (e.g., RGB)
H = 20 # Height
W = 30 # Width
embed_dim = C
L = H * W
# rgb = torch.randn(B, C, H, W)
# eve = torch.randn(B, C, H, W)
x1 = torch.randn(B, L, C)
x2 = torch.randn(B, L, C)


In [ ]:
out = ffm(x1, x2, (H, W))
print(out.shape)

In [ ]:
p = (2, 3)
p = p * 2
print(p)

In [ ]:
class CrossPath(nn.Module):
    def __init__(self, dim, reduction=1, num_heads=None, norm_layer=nn.LayerNorm):
        super().__init__()
        self.cross_attn = CrossAttention(dim // reduction, num_heads=num_heads)
        # self.end_proj1 = nn.Linear(dim // reduction, dim)
        # self.end_proj2 = nn.Linear(dim // reduction, dim)
        # self.end_proj3 = nn.Linear(dim // reduction * 2, dim)
        # self.norm1 = norm_layer(dim)
        # self.norm2 = norm_layer(dim)

    def forward(self, x1, x2):
        v1, v2 = self.cross_attn(x1, x2)
        # out_x1 = self.norm1(x1 + v1)
        # out_x2 = self.norm2(x2 + v2)
        # out = self.end_proj3(torch.cat((out_x1, out_x2), dim=-1))
        return v1, v2
        return out

cro = CrossPath(dim=1024, num_heads=16, reduction=1)
count_parameters(cro)

m = nn.Conv2d(
    in_channels=2048,
    out_channels=1024,
    kernel_size=1,
)

In [ ]:


def token2feature(tokens, patch_grid_size):
    """add token transfer to feature"""
    B, L, D = tokens.shape
    # H = W = int(L**0.5)
    H, W = patch_grid_size[0], patch_grid_size[1]
    x = tokens.permute(0, 2, 1).view(B, D, H, W).contiguous()
    return x


def feature2token(x):
    B, C, H, W = x.shape
    L = W * H
    tokens = x.view(B, C, L).permute(0, 2, 1).contiguous()
    return tokens

ffm = FeatureFusionModule(dim=1024, num_heads=16, reduction=1)
# count_parameters(ffm)
# count_parameters(ffm.cross)
count_parameters(ffm.channel_emb)

In [ ]:
B = 4  # Batch size
C = 1024  # Channels (e.g., RGB)
H = 50 # Height
W = 50 # Width
N = 100
embed_dim = C
rgb = torch.randn(B, C, H, W)
eve = torch.randn(B, C, H, W)

In [ ]:
from model.epde.prompt_module import FeatureFusionModule
from model.epde.prompt_module import FeatureRectifyModule
embed_dim = 1024
upscale_factor = 2

dim = embed_dim // (upscale_factor * upscale_factor)
ffm = FeatureFusionModule(dim=dim, reduction=1, num_heads=16)
frf = FeatureRectifyModule(dim=dim, reduction=1)
pixel_shuffle = nn.PixelShuffle(upscale_factor=upscale_factor)

print(rgb.shape)

rgb = pixel_shuffle(rgb)
eve = pixel_shuffle(eve)
print(rgb.shape)

# merge = ffm(rgb, eve)
# print(merge.shape)
count_parameters(ffm)
count_parameters(frf)

In [ ]:
merge = ChannelEmbed(C * 2, C, reduction=1)
count_parameters(merge.residual)
count_parameters(merge)

print(rgb.shape)
cat_fea = torch.cat((rgb, eve), dim=-1)
print(cat_fea.shape)
fea = merge(cat_fea, H=10, W=10)
print(fea.shape)
token = feature2token(fea)
print(token.shape)

In [ ]:
in_channels = 2048
out_channels = 1024
reduction = 1
m = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
count_parameters(m)
m = nn.Conv2d(in_channels, out_channels // reduction, kernel_size=1, bias=True)
count_parameters(m)
m = nn.Conv2d(
    out_channels // reduction,
    out_channels // reduction,
    kernel_size=3,
    stride=1,
    padding=1,
    bias=True,
    groups=out_channels // reduction,
)
count_parameters(m)
m = nn.Conv2d(out_channels // reduction, out_channels, kernel_size=1, bias=True)
count_parameters(m)

In [ ]:
from model.depth_anything_v2.dinov2_layers.patch_embed import PatchEmbed
img_size = (H, W)
patch_size = (5, 5)
patch_grid_size = (H // patch_size[0], W // patch_size[1])

img = torch.randn(B, 3, H, W)
patch_embed = PatchEmbed(
    img_size=img_size,
    patch_size=patch_size,
    in_chans=3,
    embed_dim=embed_dim,
)
tokens = patch_embed(img)
print(tokens.shape)
fea = token2feature(tokens=tokens, patch_grid_size=patch_grid_size)
print(fea.shape)

print(rgb.shape)
rgb = rgb.flatten(2).transpose(1, 2)
print(rgb.shape)

#### MM-SAM Selective Fusion Gate

In [ ]:
from typing import Union, List
import math
import torch.nn
import torch.nn.functional as F

from torch import nn
from torch.nn import Module


def feature_fusion(
    joint_feats: torch.Tensor, joint_feat_masks: torch.Tensor
) -> torch.Tensor:
    # (B, N_patch, N_filter, feat_num)
    joint_feat_masks = F.softmax(joint_feat_masks, dim=-1)
    # (B, N_patch, N_filter, feat_num) -> (N_filter, B, N_patch, feat_num) -> (N_filter, B, N_patch, 1, feat_num)
    joint_feat_masks = joint_feat_masks.permute(2, 0, 1, 3).unsqueeze(-2)

    # (B, N_patch, C, feat_num) -> (1, B, N_patch, C, feat_num)
    joint_feats = joint_feats.unsqueeze(0)

    # (N_filter, B, N_patch, 1, feat_num) * (1, B, N_patch, C, feat_num) -> (N_filter, B, N_patch, C, feat_num)
    joint_feats = joint_feats * joint_feat_masks
    # weighted sum: (N_filter, B, N_patch, C, feat_num) -> (N_filter, B, N_patch, C)
    joint_feats = torch.sum(joint_feats, dim=-1)
    # filter average: (N_filter, B, N_patch, C) -> (B, N_patch, C)
    joint_feats = torch.mean(joint_feats, dim=0)
    return joint_feats


def feature_fusion_by_rgb_x(
    rgb_feats: torch.Tensor,
    x_feats: torch.Tensor,
    rgb_feat_masks: torch.Tensor,
    x_feat_masks: torch.Tensor,
):
    # (B, N_patch, N_filter, 1) * 2 -> (B, N_patch, N_filter, 2)
    joint_feat_masks = torch.cat(
        (rgb_feat_masks.unsqueeze(-1), x_feat_masks.unsqueeze(-1)), dim=-1
    )
    # (B, N_patch, C, 1) * 2 -> (B, N_patch, C, 2)
    joint_feats = torch.cat((rgb_feats.unsqueeze(-1), x_feats.unsqueeze(-1)), dim=-1)
    return feature_fusion(joint_feats, joint_feat_masks)


class SelectiveFusionGate(Module):
    def __init__(
        self,
        in_channels_per_feat: int = 256,
        filter_num: int = 1,
        feat_num: int = 2,
        intermediate_channels: Union[int, List[int], None] = None,
        filter_type: str = "linear",
    ):
        super().__init__()
        if intermediate_channels is None:
            intermediate_channels = []
        if isinstance(intermediate_channels, int):
            intermediate_channels = [intermediate_channels]

        in_channels = in_channels_per_feat * feat_num
        if filter_type == "linear":
            filter_bank, layer_in_features = [], in_channels
            for layer_out_features in intermediate_channels:
                filter_bank.extend(
                    [
                        nn.Linear(
                            in_features=layer_in_features,
                            out_features=layer_out_features,
                        ),
                        nn.GELU(),
                    ]
                )
                layer_in_features = layer_out_features
            filter_bank.append(
                nn.Linear(
                    in_features=layer_in_features, out_features=filter_num * feat_num
                )
            )
            self.filter_bank = nn.Sequential(*filter_bank)

        elif filter_type == "conv2d":
            self.filter_bank = Conv2dFilterBank(
                in_channels=in_channels,
                filter_num=filter_num * feat_num,
                intermediate_channels=intermediate_channels,
            )

        else:
            raise NotImplementedError(f"Filter type {filter_type} not implemented!")

    def forward(self, feat_list: List[torch.Tensor]) -> (torch.Tensor, torch.Tensor):
        # (B, C, H, W, feat_num)
        joint_feats = torch.stack(feat_list, dim=-1)

        # (B, C, H, W, feat_num) -> (B, C, N_patch, feat_num) -> (B, N_patch, feat_num, C)
        # Note: feat_num should be in front of C for correct concatenation
        bs, c, h, w, feat_num = joint_feats.shape
        joint_feats = joint_feats.flatten(2, 3).permute(0, 2, 3, 1)
        # (B, N_patch, feat_num, C) -> (B, N_patch, C * feat_num)
        joint_feats = joint_feats.reshape(bs, h * w, -1)

        # filter mask generation: (B, N_patch, C * feat_num) -> (B, N_patch, N_filter * feat_num)
        joint_feat_masks = self.filter_bank(joint_feats)
        # (B, N_patch, N_filter * feat_num) -> (B, N_patch, N_filter, feat_num)
        joint_feat_masks = joint_feat_masks.reshape(bs, h * w, -1, feat_num)

        # (B, N_patch, C * feat_num) -> (B, N_patch, feat_num, C) -> (B, N_patch, C, feat_num)
        joint_feats = joint_feats.reshape(bs, h * w, feat_num, -1).permute(0, 1, 3, 2)

        # (B, N_patch, C)
        joint_feats = feature_fusion(joint_feats, joint_feat_masks)

        # back to patch style: (B, N_patch, C) -> (B, C, H, W)
        joint_feats = joint_feats.permute(0, 2, 1).reshape(bs, c, h, w)
        # (B, N_patch, N_filter, feat_num) -> (B, H, W, N_filter, feat_num)
        filter_num = joint_feat_masks.size(-2)
        joint_feat_masks = joint_feat_masks.reshape(bs, h, w, filter_num, feat_num)
        return joint_feats, joint_feat_masks


class Conv2dFilterBank(Module):
    def __init__(
        self,
        in_channels: int = 256,
        filter_num: int = 1,
        kernel_size: int = 3,
        intermediate_channels: Union[int, List[int], None] = None,
    ):
        super().__init__()
        assert kernel_size % 2 == 1, "kernel_size should be odd number!"
        padding = kernel_size // 2

        conv_filter_bank, layer_in_channels = [], in_channels
        print(layer_in_channels)
        for layer_out_channels in intermediate_channels:
            conv_filter_bank.extend(
                [
                    nn.Conv2d(
                        in_channels=layer_in_channels,
                        out_channels=layer_out_channels,
                        kernel_size=kernel_size,
                        padding=padding,
                    ),
                    nn.GELU(),
                ]
            )
            layer_in_channels = layer_out_channels
        conv_filter_bank.append(
            nn.Conv2d(
                in_channels=layer_in_channels,
                out_channels=filter_num,
                kernel_size=kernel_size,
                padding=padding,
            )
        )
        self.conv_filter_bank = nn.Sequential(*conv_filter_bank)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # get the height and width for reshaping
        side_len = int(math.sqrt(x.shape[-2]))
        # (B, N_patch, C) -> (B, C, N_patch) -> (B, C, H, W)
        x = x.permute(0, 2, 1).reshape(x.shape[0], -1, side_len, side_len)
        # (B, C, H, W) -> (B, N_filter, H, W)
        x = self.conv_filter_bank(x)
        # (B, N_filter, H, W) -> (B, N_filter, N_patch) -> (B, N_patch, N_filter)
        x = x.flatten(2).permute(0, 2, 1)
        return x

In [ ]:
fusion_module = SelectiveFusionGate(
    in_channels_per_feat=1024,
    filter_num=1,
    intermediate_channels=1024,
    filter_type="conv2d",
)
count_parameters(fusion_module)
m = nn.Conv2d(
    in_channels=512,
    out_channels=256,
    kernel_size=1,
    padding=(1//2),
)
count_parameters(m)

#### Test Feature Fusion

In [ ]:
from model.epde.prompt_module import Prompt_block, Fovea
from model.epde.utils import token2feature, feature2token
from util.vis import vis_feature_var, vis_depth_map
import numpy as np
import torch
import torch.nn as nn

import matplotlib
import matplotlib.pylab as plt

In [ ]:
# def vis_depth_map(dep, show_colorbar=False, cmap_name="Spectral", save_fig=False, save_path="test.png"):
#     cmap = matplotlib.colormaps.get_cmap(cmap_name)

#     fig, ax = plt.subplots()
#     fig.subplots_adjust(left=0, right=1, top=1, bottom=0)
#     plt.axis("off")

#     im = ax.imshow(dep, cmap=cmap, extent=[0, dep.shape[1], 0, dep.shape[0]])
#     if show_colorbar:
#         plt.colorbar(im)

#     dpi = fig.get_dpi()
#     fig.set_size_inches(dep.shape[1] / dpi, dep.shape[0] / dpi)

#     if save_fig:
#         plt.savefig(save_path)
#     plt.show()

In [ ]:
# EventScape
# patch_size = (38, 19) # 266
# patch_size = (50, 25) # 350
# patch_size = (74, 37) # 518 
# patch_size = (148, 74) # 1036

# MVSEC
# patch_size = (25, 19) # 266
# patch_size = (49, 37) # 518
patch_size = (98, 74) # 1036

# DSEC
# patch_size = (99, 74)# 1036
patch_size = (patch_size[1], patch_size[0])

In [ ]:
fovea = Fovea()
pixel_shuffle = nn.PixelShuffle(upscale_factor=2)
pixel_unshuffle = nn.PixelUnshuffle(downscale_factor=2)

In [ ]:
path = "/data/coding/code/da2-prompt-tuning/vis_feature_maps/align_log_l1andfea_eventscape_mxsec_1036_2_lora/fea/04056_0.png.npy"
tokens = torch.from_numpy(np.load(path)).unsqueeze(0)
feas = token2feature(tokens=tokens, patch_grid_size=patch_size)
var = torch.var(feas, dim=1, keepdim=True).squeeze().numpy()
vis_depth_map(var, cmap_name="viridis")

# feas = fovea(feas)
# var = torch.var(feas, dim=1, keepdim=True).squeeze().numpy()
# vis_depth_map(var, cmap_name="viridis")

print(feas.shape)
feas = pixel_shuffle(feas)
print(feas.shape)
var = torch.var(feas, dim=1, keepdim=True).squeeze().numpy()
vis_depth_map(var, cmap_name="viridis")

print(feas.shape)
feas = pixel_unshuffle(feas)
print(feas.shape)
var = torch.var(feas, dim=1, keepdim=True).squeeze().numpy()
vis_depth_map(var, cmap_name="viridis")
# tokens = feature2token(feas)
# print(tokens.shape)
# vis_feature_var(tokens, patch_size=patch_size)

In [ ]:
import torch
import numpy as np
from pprint import pprint

In [ ]:
metrics_keywords = [
    f"_abs_rel_diff",
    f"_squ_rel_diff",
    f"_RMS_linear",
    f"_RMS_log",
    f"_SILog",
    f"_mean_target_depth",
    f"_median_target_depth",
    f"_mean_prediction_depth",
    f"_median_prediction_depth",
    f"_mean_depth_error",
    f"_median_diff",
    f"_threshold_delta_1.25",
    f"_threshold_delta_1.25^2",
    f"_threshold_delta_1.25^3",
    f"_10_mean_target_depth",
    f"_10_median_target_depth",
    f"_10_mean_prediction_depth",
    f"_10_median_prediction_depth",
    f"_10_abs_rel_diff",
    f"_10_squ_rel_diff",
    f"_10_RMS_linear",
    f"_10_RMS_log",
    f"_10_SILog",
    f"_10_mean_depth_error",
    f"_10_median_diff",
    f"_10_threshold_delta_1.25",
    f"_10_threshold_delta_1.25^2",
    f"_10_threshold_delta_1.25^3",
    f"_20_abs_rel_diff",
    f"_20_squ_rel_diff",
    f"_20_RMS_linear",
    f"_20_RMS_log",
    f"_20_SILog",
    f"_20_mean_target_depth",
    f"_20_median_target_depth",
    f"_20_mean_prediction_depth",
    f"_20_median_prediction_depth",
    f"_20_mean_depth_error",
    f"_20_median_diff",
    f"_20_threshold_delta_1.25",
    f"_20_threshold_delta_1.25^2",
    f"_20_threshold_delta_1.25^3",
    f"_30_abs_rel_diff",
    f"_30_squ_rel_diff",
    f"_30_RMS_linear",
    f"_30_RMS_log",
    f"_30_SILog",
    f"_30_mean_target_depth",
    f"_30_median_target_depth",
    f"_30_mean_prediction_depth",
    f"_30_median_prediction_depth",
    f"_30_mean_depth_error",
    f"_30_median_diff",
    f"_30_threshold_delta_1.25",
    f"_30_threshold_delta_1.25^2",
    f"_30_threshold_delta_1.25^3",
]

In [ ]:
def add_to_metrics(
    idx,
    metrics,
    target_,
    prediction_,
    mask,
    event_frame=None,
    prefix="",
    debug=False,
    output_folder=None,
):
    if len(metrics) == 0:
        metrics = {k: 0 for k in metrics_keywords}

    prediction_mask = (prediction_ > 0) & (
        prediction_ <= np.amax(target_[~np.isnan(target_)])
    )
    depth_mask = (target_ > 0) & (
        target_ <= np.amax(target_[~np.isnan(target_)])
    )  # make (target> 3) for mvsec might drives
    mask = mask & depth_mask & prediction_mask
    eps = 1e-5

    target = target_[
        mask
    ]  # np.where(mask, target_, np.max(target_[~np.isnan(target_)]))# target_[mask] but without lossing shape
    prediction = prediction_[
        mask
    ]  # np.where(mask, prediction_, np.max(target_[~np.isnan(target_)]))# prediction_[mask] but without lossing shape

    # thresholds
    # ratio = np.max(np.stack([target/(prediction+eps),prediction/(target+eps)]), axis=0)
    ratio = np.maximum((target / (prediction + eps)), (prediction / (target + eps)))

    new_metrics = {}
    new_metrics[f"{prefix}threshold_delta_1.25"] = np.mean(ratio <= 1.25)
    new_metrics[f"{prefix}threshold_delta_1.25^2"] = np.mean(ratio <= 1.25**2)
    new_metrics[f"{prefix}threshold_delta_1.25^3"] = np.mean(ratio <= 1.25**3)

    # abs diff
    # log_diff = np.log(target+eps)-np.log(prediction+eps)
    log_diff = np.log(prediction + eps) - np.log(target + eps)
    # log_diff = np.abs(log_target - log_prediction)
    abs_diff = np.abs(target - prediction)

    new_metrics[f"{prefix}abs_rel_diff"] = (abs_diff / (target + eps)).mean()
    # new_metrics[f"{prefix}squ_rel_diff"] = (abs_diff**2 / (target**2 + eps)).mean()
    new_metrics[f"{prefix}squ_rel_diff"] = (abs_diff**2 / (target + eps)).mean()
    new_metrics[f"{prefix}RMS_linear"] = np.sqrt((abs_diff**2).mean())
    new_metrics[f"{prefix}RMS_log"] = np.sqrt((log_diff**2).mean())
    new_metrics[f"{prefix}SILog"] = (log_diff**2).mean() - (log_diff.mean()) ** 2
    new_metrics[f"{prefix}mean_target_depth"] = target.mean()
    new_metrics[f"{prefix}median_target_depth"] = np.median(target)
    new_metrics[f"{prefix}mean_prediction_depth"] = prediction.mean()
    new_metrics[f"{prefix}median_prediction_depth"] = np.median(prediction)
    new_metrics[f"{prefix}mean_depth_error"] = abs_diff.mean()
    new_metrics[f"{prefix}median_diff"] = np.abs(
        np.median(target) - np.median(prediction)
    )

    for k, v in new_metrics.items():
        metrics[k] += v

    return metrics

In [ ]:
dataset2params = {
    "dense": {"clip_distance": 1000.0, "reg_factor": 6.2044},
    "mvsec": {"clip_distance": 80.0, "reg_factor": 3.70378},
    "eventscape": {"clip_distance": 1000.0, "reg_factor": 5.7},
}
reg_factor = dataset2params["mvsec"]["reg_factor"]
max_depth = dataset2params["mvsec"]["clip_distance"]
crop_ymax = 260
nan_mask = True

p_file = "/home/sph/event/da2-prompt-tuning/results/test/epde_metric_80_mvsec_2_decoder_116131204_mvsec_night1_7/npy/00020.npy"
t_file = "/data_nvme/sph/mvsec_processed/outdoor_night1/depths/00020.npy"

In [ ]:
min_depth = np.exp(-1 * reg_factor) * max_depth

# Read absolute scale ground truth
target_depth = np.load(t_file)
# Crop depth height according to argument
target_depth = target_depth[: crop_ymax]

# Read predicted depth data
predicted_depth = np.load(p_file)
# Crop depth height according to argument
predicted_depth = predicted_depth[: crop_ymax]

if nan_mask:
    non_nan_mask = ~np.isnan(target_depth)
    target_depth = target_depth[non_nan_mask]
    predicted_depth = predicted_depth[non_nan_mask]

assert target_depth.ndim <= 2
assert predicted_depth.ndim <= 2

# Flat numpy arrary
if target_depth.ndim == 2:
    target_depth = target_depth.ravel()
    predicted_depth = predicted_depth.ravel()

assert predicted_depth.shape == target_depth.shape

depth_mask = np.ones_like(target_depth) > 0
# Clip
target_depth = np.clip(target_depth, min_depth, max_depth)
predicted_depth = np.clip(predicted_depth, min_depth, max_depth)

In [ ]:
target = torch.from_numpy(np.load(t_file))
pred = torch.from_numpy(np.load(p_file))

# non_nan_mask = torch.isfinite(target)
# target = target[non_nan_mask]
# pred = pred[non_nan_mask]

# min_depth = torch.exp(-1 * torch.tensor(reg_factor)) * torch.tensor(max_depth)

# target = torch.clamp(target, min=min_depth, max=max_depth)
# pred = torch.clamp(pred, min=min_depth, max=max_depth)

In [ ]:
# target_depth = torch.from_numpy(target_depth)
# predicted_depth = torch.from_numpy(predicted_depth)

tar_diff = target_depth - target
pre_diff = predicted_depth - pred

print(tar_diff.abs().max(), pre_diff.abs().max())

In [ ]:
metrics = {}
metrics = add_to_metrics(
            0,
            metrics,
            target_depth,
            predicted_depth,
            depth_mask,
            event_frame=None,
            prefix="_",
            debug=False,
            output_folder=None,
        )
print(metrics)

In [ ]:
print(pred.min(), target.min(), pred.max(), target.max())

non_nan_mask = torch.isfinite(target)
target = target[non_nan_mask]
pred = pred[non_nan_mask]
print(pred.min(), target.min(), pred.max(), target.max())

reg_factor = dataset2params["mvsec"]["reg_factor"]
max_depth = dataset2params["mvsec"]["clip_distance"]
min_depth = torch.exp(-1 * torch.tensor(reg_factor)) * torch.tensor(max_depth)

target = torch.clamp(target, min=min_depth, max=max_depth)
pred = torch.clamp(pred, min=min_depth, max=max_depth)

thresh = torch.max((target / pred), (pred / target))

d1 = torch.sum(thresh < 1.25).float() / len(thresh)
d2 = torch.sum(thresh < 1.25 ** 2).float() / len(thresh)
d3 = torch.sum(thresh < 1.25 ** 3).float() / len(thresh)

diff = pred - target
diff_log = torch.log(pred) - torch.log(target)

abs_rel = torch.mean(torch.abs(diff) / target)
sq_rel = torch.mean(torch.pow(diff, 2) / target)

rmse = torch.sqrt(torch.mean(torch.pow(diff, 2)))
rmse_log = torch.sqrt(torch.mean(torch.pow(diff_log , 2)))

log10 = torch.mean(torch.abs(torch.log10(pred) - torch.log10(target)))
silog = torch.sqrt(torch.pow(diff_log, 2).mean() - 0.5 * torch.pow(diff_log.mean(), 2))

ms = {'d1': d1.item(), 'd2': d2.item(), 'd3': d3.item(), 'abs_rel': abs_rel.item(), 'sq_rel': sq_rel.item(), 
        'rmse': rmse.item(), 'rmse_log': rmse_log.item(), 'log10':log10.item(), 'silog':silog.item()}
print(ms)

In [ ]:
from util.metric import eval_depth
target = torch.from_numpy(np.load(t_file))
pred = torch.from_numpy(np.load(p_file))
assert pred.shape == target.shape
print(pred.min(), pred.max())

ms = eval_depth(pred, target, dataset="mvsec")
print(ms)

In [ ]:
def prepare_depth(depth, reg_factor, d_min, d_max):
    # Normalize depth
    depth = np.clip(depth, d_min, d_max)
    depth = depth / d_max
    depth = np.log(depth) / reg_factor + 1.0
    depth = depth.clip(0.0, 1.0)
    return depth

In [ ]:
import glob
from os.path import join
import tqdm
import torch
import numpy as np
from util.metric import eval_depth, convert_nl2abs_depth

reg_factor, d_min, d_max = 3.70378, 1.97041, 80
results = {
    "d1": torch.tensor([0.0]).cuda(),
    "d2": torch.tensor([0.0]).cuda(),
    "d3": torch.tensor([0.0]).cuda(),
    "abs_rel": torch.tensor([0.0]).cuda(),
    "sq_rel": torch.tensor([0.0]).cuda(),
    "rmse": torch.tensor([0.0]).cuda(),
    "rmse_log": torch.tensor([0.0]).cuda(),
    "log10": torch.tensor([0.0]).cuda(),
    "silog": torch.tensor([0.0]).cuda(),
}
predictions_dataset = "/home/sph/event/da2-prompt-tuning/results/test/epde_nl_mvsec_2_decoder_mvsec_night1_3/npy"
target_dataset = "/data_nvme/sph/mvsec_processed/outdoor_night1/depths"
prediction_files = sorted(glob.glob(join(predictions_dataset, "*.npy")))
target_files = sorted(glob.glob(join(target_dataset, "*.npy")))

num_it = len(target_files)
nsamples = 0
for idx in tqdm.tqdm(range(num_it)):
    p_file, t_file = prediction_files[idx], target_files[idx]

    # target = torch.from_numpy(np.load(t_file))
    target = np.load(t_file)
    target = prepare_depth(target, reg_factor=reg_factor, d_min=d_min, d_max=d_max)
    target = convert_nl2abs_depth(target, clip_distance=d_max, reg_factor=reg_factor)
    target = torch.from_numpy(target)
    pred = torch.from_numpy(np.load(p_file))

    cur_results = eval_depth(pred, target, dataset="mvsec")
    for k in results.keys():
        results[k] += cur_results[k]
    nsamples += 1

cur_results = {}
for k in results.keys():
    cur_results[k] = (results[k] / nsamples).item()
print(cur_results)

In [ ]:
from model.epde_modal_metric import EPDE
import torch
from model.epde.utils import clean_pretrained_weight

In [ ]:
p = "/home/sph/event/da2-prompt-tuning/exp/epde_nl_mvsec_2_decoder_20250116_222954/abs_rel-0.26239070296287537-6.pth"

model = EPDE(
    model_name="vitl",
    max_depth=1,
    event_voxel_chans=3,
    return_feature=False
)

latest_ckp = torch.load(p, map_location="cpu")
latest_ckp = latest_ckp["model"]
model.load_state_dict(latest_ckp, strict=True)

In [ ]:
state_dict = model.module.state_dict()
print(type(model))
print(state_dict.keys())

In [ ]:
import torch

In [ ]:
from util.utils import compare_model_weights
p1 = "/home/sph/event/da2-prompt-tuning/exp/epde_nl_mvsec_2_decoder_20250117_165144/abs_rel-0.5710307359695435-0.pth"
p2 = "/home/sph/event/da2-prompt-tuning/exp/epde_nl_mvsec_2_decoder_20250117_165730/abs_rel-0.5476359128952026-0.pth"

c1 = torch.load(p1, map_location="cpu")
c1 = c1["model"]

# compare_model_weights(c1, c2, "test.txt")

In [ ]:
import torch
from util.utils import compare_model_weights
from model.epde_modal_metric import EPDE
import copy

p1 = "/home/sph/event/da2-prompt-tuning/exp/epde_nl_mvsec_2_decoder_20250117_165144/abs_rel-0.5710307359695435-0.pth"

c1 = torch.load(p1, map_location="cpu")
c1 = c1["model"]

model = EPDE(
    model_name="vitl",
    max_depth=1,
    event_voxel_chans=3,
    return_feature=False
)

model.load_state_dict(c1, strict=True)
c2 = model.state_dict()

c11 = copy.deepcopy(c1)
c22 = copy.deepcopy(c2)
# model.eval()
compare_model_weights(c11, c22, "test2.txt")

In [ ]:
def compare_model_weights(state_dict1, state_dict2):
    # Compare the differences between two state_dict objects 
    # (whether they have the same keys and the same values).
    keys1 = set(state_dict1.keys())
    keys2 = set(state_dict2.keys())

    missing_in_model1 = keys2 - keys1  # Keys present in model2 but not in model1
    missing_in_model2 = keys1 - keys2  # Keys present in model1 but not in model2

    all_match = True

    if missing_in_model1 or missing_in_model2:
        all_match = False
        print("State dict keys do not match.\n")

        if missing_in_model1:
            print(f"Keys missing in model1: {missing_in_model1}\n")

        if missing_in_model2:
            print(f"Keys missing in model2: {missing_in_model2}\n")
        
    common_keys = keys1.intersection(keys2)
    for key in common_keys:
        if not torch.allclose(state_dict1[key], state_dict2[key]):
            all_match = False
            print(f"Weight mismatch found at layer: {key}\n")
            print(f"Model 1 tensor: {state_dict1[key]}\n")
            print(f"Model 2 tensor: {state_dict2[key]}\n")
            print("-" * 80 + "\n")

    if all_match:
            print("All weights match.")
    return all_match


checkpoint_path = "..."
# This checkpoint contains all the weights of the model, 
# including those belonging to LoRA and those of the pre-trained model.
ckp = torch.load(checkpoint_path, map_location="cpu")

# The model contains layers of lora.Linear().
model = Model(...)
# Loading weights in training mode may lead to anomalies.
model.train()
model.load_state_dict(ckp, strict=True)
ckp2= model.state_dict()

# This is very strange. If I execute model.eval(), 
# ckp and ckp2 are different; if I remove it, they are the same.
model.eval()
compare_model_weights(ckp, ckp2)

In [ ]:
def add_gaussian_noise(image, mean=0, sigma=0.05):
    """
    Add Gaussian noise to multi-channel data with automatic range adaptation.
    Handles input shape [B, C, H, W] where:
    - B: Batch size
    - C: Channel number
    - H: Height
    - W: Width
    
    The noise level (sigma) should be specified in normalized [0,1] range.
    """
    # Store original dtype and convert to float32 for calculations
    original_dtype = image.dtype
    image = image.astype(np.float32)
    
    # Calculate min/max per sample and channel while keeping dimensions
    # data_min = np.min(image, axis=(2, 3), keepdims=True)
    # data_max = np.max(image, axis=(2, 3), keepdims=True)
    data_min = np.min(image)
    data_max = np.max(image)
    
    # Normalize to [0, 1] range with epsilon to avoid division by zero
    normalized = (image - data_min) / (data_max - data_min + 1e-8)
    
    # Generate Gaussian noise in normalized space
    noise = np.random.normal(mean, sigma, image.shape)
    print(f"noise: {noise.min()}, {noise.max()}")
    # Add noise and clip to valid normalized range
    noisy_normalized = np.clip(normalized + noise, 0, 1)
    
    # Restore original data range
    noisy_image = noisy_normalized * (data_max - data_min) + data_min
    
    # Convert back to original data type
    return noisy_image.astype(original_dtype)

def change_brightness(image, factor=0.5):
    image_float = image.astype(np.float32) * factor
    return image_float.round().clip(0, 255).astype(np.uint8)


In [ ]:
from util.vis import vis_voxelgrid
import numpy as np
import cv2
import matplotlib.pyplot as plt
from dataset.transform import Image_Corruption

imgcor = Image_Corruption(
            mask_pro=0.5, cor_types=["blur", "mask", "overexpose"]
        )
voxcor = Image_Corruption(
            mask_pro=0.5, cor_types=["blur", "mask"]
        )

In [ ]:
p = "/data_nvme/sph/EventScape_processed/Town01-03_train/Town01/sequence_0/events/voxels/01_000_0010_events.npy"
# p = "/data_nvme/sph/mvsec_processed/outdoor_day1/voxels/00007.npy"
vox = np.load(p)
print(vox.min(), vox.max(), vox.shape)
vis_voxelgrid(vox)
vox = vox.transpose(1, 2, 0)

In [ ]:
vox = voxcor(image=vox)
print(vox.shape, vox.min(), vox.max())
vox = vox.transpose(2, 0, 1)
vis_voxelgrid(vox)

In [ ]:
img_path = "/data_nvme/sph/EventScape_processed/Town01-03_train/Town01/sequence_2/rgb/data/01_002_0005_image.png"
image = cv2.imread(img_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
print(image.shape, image.min(), image.max(), image.dtype)
plt.imshow(image)
plt.axis('off')
plt.show()

In [ ]:
image = change_brightness(image=image, factor=1.5)
print(image.shape, image.dtype, image.min(), image.max())
plt.imshow(image)
plt.axis('off')
plt.show()

In [ ]:
image = imgcor(image=image)
print(image.shape, image.min(), image.max())
plt.imshow(image)
plt.axis('off')
plt.show()

In [ ]:
a = (1, 1)
import random
b = random.uniform(a[0], a[1])
print(b)

In [ ]:
from dataset.transform import Image_Corruption
import cv2
#  ["blur", "overexpose", "mask"])
img_cor1 = Image_Corruption(mask_pro=1, num_masks=1, mask_radio=0.3, cor_types=["overexpose"], brightness_range=[0.5, 1.5])
img_cor2 = Image_Corruption(mask_pro=1, num_masks=1, mask_radio=0.3, cor_types=["blur"], brightness_range=[0.5, 1.5])
img_cor3 = Image_Corruption(mask_pro=1, num_masks=1, mask_radio=0.3, cor_types=["mask"], brightness_range=[0.5, 1.5])

In [ ]:
ps = [
    "/data_nvme/sph/EventScape_processed/Town01-03_train/Town01/sequence_2/rgb/data/01_002_0004_image.png",
    # "/data_nvme/sph/EventScape_processed/Town01-03_train/Town03/sequence_7/rgb/data/03_007_0003_image.png",
    "/data_nvme/sph/EventScape_processed/Town01-03_train/Town01/sequence_5/rgb/data/01_005_0004_image.png",
    "/data_nvme/sph/EventScape_processed/Town01-03_train/Town02/sequence_20/rgb/data/02_020_0025_image.png",
]

for i in range(len(ps)):
    img_path = ps[i]
    
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    image = img_cor1(image)
    image = img_cor2(image)
    image = img_cor3(image)
    
    output_path = f'{i}_img_cor.png'

    cv2.imwrite(output_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

In [ ]:
vox_cor1 = Image_Corruption(mask_pro=1, num_masks=1, mask_radio=0.3, cor_types=["mask"])
vox_cor2 = Image_Corruption(mask_pro=1, num_masks=1, mask_radio=0.3, cor_types=["blur"])

In [ ]:
for i in range(3):
    img_path = f"/home/sph/event/da2-prompt-tuning/dataset/{i}_e.png"
    
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    image = vox_cor1(image)
    image = vox_cor2(image)
    
    output_path = f'{i}_e_cor.png'

    cv2.imwrite(output_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

In [ ]:
# Author: Bingxin Ke
# Last modified: 2024-01-11

import numpy as np
import torch


def align_depth_least_square(
    gt_arr: np.ndarray,
    pred_arr: np.ndarray,
    valid_mask_arr: np.ndarray,
    return_scale_shift=True,
    max_resolution=None,
):
    ori_shape = pred_arr.shape  # input shape

    gt = gt_arr.squeeze()  # [H, W]
    pred = pred_arr.squeeze()
    valid_mask = valid_mask_arr.squeeze()

    # Downsample
    if max_resolution is not None:
        scale_factor = np.min(max_resolution / np.array(ori_shape[-2:]))
        if scale_factor < 1:
            downscaler = torch.nn.Upsample(scale_factor=scale_factor, mode="nearest")
            gt = downscaler(torch.as_tensor(gt).unsqueeze(0)).numpy()
            pred = downscaler(torch.as_tensor(pred).unsqueeze(0)).numpy()
            valid_mask = (
                downscaler(torch.as_tensor(valid_mask).unsqueeze(0).float())
                .bool()
                .numpy()
            )

    assert (
        gt.shape == pred.shape == valid_mask.shape
    ), f"{gt.shape}, {pred.shape}, {valid_mask.shape}"

    gt_masked = gt[valid_mask].reshape((-1, 1))
    pred_masked = pred[valid_mask].reshape((-1, 1))

    # numpy solver
    _ones = np.ones_like(pred_masked)
    A = np.concatenate([pred_masked, _ones], axis=-1)
    X = np.linalg.lstsq(A, gt_masked, rcond=None)[0]
    scale, shift = X

    aligned_pred = pred_arr * scale + shift

    # restore dimensions
    aligned_pred = aligned_pred.reshape(ori_shape)

    if return_scale_shift:
        return aligned_pred, scale, shift
    else:
        return aligned_pred


# ******************** disparity space ********************
def depth2disparity(depth, return_mask=False):
    if isinstance(depth, torch.Tensor):
        disparity = torch.zeros_like(depth)
    elif isinstance(depth, np.ndarray):
        disparity = np.zeros_like(depth)
    non_negtive_mask = depth > 0
    disparity[non_negtive_mask] = 1.0 / depth[non_negtive_mask]
    if return_mask:
        return disparity, non_negtive_mask
    else:
        return disparity


def disparity2depth(disparity, **kwargs):
    return depth2disparity(disparity, **kwargs)

In [ ]:
alignment = ""
# Align with GT using least square
if "least_square" == alignment:
    depth_pred, scale, shift = align_depth_least_square(
        gt_arr=depth_raw,
        pred_arr=depth_pred,
        valid_mask_arr=valid_mask,
        return_scale_shift=True,
        max_resolution=alignment_max_res,
    )
elif "least_square_disparity" == alignment:
    # convert GT depth -> GT disparity
    gt_disparity, gt_non_neg_mask = depth2disparity(
        depth=depth_raw, return_mask=True
    )
    # LS alignment in disparity space
    pred_non_neg_mask = depth_pred > 0
    valid_nonnegative_mask = valid_mask & gt_non_neg_mask & pred_non_neg_mask

    disparity_pred, scale, shift = align_depth_least_square(
        gt_arr=gt_disparity,
        pred_arr=depth_pred,
        valid_mask_arr=valid_nonnegative_mask,
        return_scale_shift=True,
        max_resolution=alignment_max_res,
    )
    # convert to depth
    disparity_pred = np.clip(
        disparity_pred, a_min=1e-3, a_max=None
    )  # avoid 0 disparity
    depth_pred = disparity2depth(disparity_pred)

# Clip to dataset min max
depth_pred = np.clip(
    depth_pred, a_min=dataset.min_depth, a_max=dataset.max_depth
)

# clip to d > 0 for evaluation
depth_pred = np.clip(depth_pred, a_min=1e-6, a_max=None)


In [ ]:
import numpy as np

a = np.random.random(10)
print(a.shape)
a = a.reshape((-1, 1))
print(a.shape)

In [ ]:
import numpy as np
def prepare_depth(depth, reg_factor, d_max, eps=1e-6):
    # Normalize depth
    # depth = np.clip(depth, 0.0, d_max)
    depth = depth / d_max
    depth = np.log(depth + eps) / reg_factor + 1.0
    # depth = depth.clip(0.0, 1.0)
    return depth

In [ ]:
a = np.array((1.97, 1, 50, 80, 90), dtype=np.float32)
reg_factor = 3.70378
d_max = 80

a = prepare_depth(a, reg_factor, d_max)
print(a)

In [7]:
import cv2
import random
import numpy as np
import os

In [ ]:
class Image_Corruption(object):
    def __init__(
        self,
        # noise_pro=0.3,
        mask_pro=0.3,
        num_masks=3,
        mask_radio=0.2,
        exposure_alpha=2.5,
        exposure_beta=200,
        blur_size=25,
        blur_sigmaX=2,
        brightness_range=(1, 1),
        cor_types=["blur", "overexpose", "mask"],
    ):
        self.mask_pro = mask_pro

        self.num_masks = num_masks
        self.mask_radio = mask_radio

        self.exposure_alpha = exposure_alpha
        self.exposure_beta = exposure_beta

        self.blur_size = blur_size
        self.blur_sigmaX = blur_sigmaX

        self.brightness_range = brightness_range 
        self.cor_types = cor_types

    def generate_masks(self, image_shape, num_masks, width_ratio=0.2, height_ratio=0.2):
        assert width_ratio <= 1
        assert height_ratio <= 1

        height, width, _ = image_shape
        masks = []

        # Convert ratios to actual dimensions
        w = int(width * width_ratio)
        h = int(height * height_ratio)

        for _ in range(num_masks):
            mask = np.zeros((height, width), dtype=np.uint8)
            # Generate a rectangle mask with size based on the percentage of the image dimensions
            x = random.randint(0, width - w) if width > w else 0
            y = random.randint(0, height - h) if height > h else 0
            cv2.rectangle(mask, (x, y), (x + w, y + h), 255, -1)

            masks.append(mask)

        return masks

    def apply_gaussian_blur_region(self, image, mask, k_size=(15, 15), sigmaX=0):
        if isinstance(k_size, int):
            k_size = (k_size, k_size)
        blurred = cv2.GaussianBlur(image, k_size, sigmaX)
        return np.where(mask[..., None] == 255, blurred, image)

    def apply_overexposure_region(self, image, mask, alpha=1.5, beta=50):
        overexposed = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
        return np.where(mask[..., None] == 255, overexposed, image)

    def mask_region(self, image, mask, mask_color=(0, 0, 0)):
        colored_mask = np.zeros_like(image)
        colored_mask[:] = mask_color
        return np.where(mask[..., None] == 255, colored_mask, image)

    def change_brightness(self, image, factor=0.5):
        image_float = image.astype(np.float32) * factor
        return image_float.round().clip(0, 255).astype(np.uint8)
    
    def __call__(self, image):
        # Change Image Brightness
        brightness_factor = random.uniform(self.brightness_range[0], self.brightness_range[1])
        image = self.change_brightness(image=image, factor=brightness_factor)
        
        if random.random() <= self.mask_pro:
            masks = self.generate_masks(
                image_shape=image.shape,
                num_masks=self.num_masks,
                width_ratio=self.mask_radio,
                height_ratio=self.mask_radio,
            )
            effect_type = random.choice(self.cor_types)
            for mask in masks:
                if effect_type == "blur":
                    image = self.apply_gaussian_blur_region(
                        image,
                        mask,
                        k_size=self.blur_size,
                        sigmaX=self.blur_sigmaX,
                    )
                elif effect_type == "overexpose":
                    image = self.apply_overexposure_region(
                        image, mask, alpha=self.exposure_alpha, beta=self.exposure_beta
                    )
                elif effect_type == "mask":
                    image = self.mask_region(image, mask, mask_color=(0, 0, 0))
                    # image = self.mask_region(image, mask, mask_color=(255, 255, 255))

        return image

In [ ]:
image_cor_process = Image_Corruption()
voxel_cor_process = Image_Corruption()

In [ ]:
p = "/home/sph/event/da2-prompt-tuning/dataset/splits/dense/test.txt"
with open(p, 'r') as f:
    lines = f.readlines()

for line in lines:
    p = line.strip()
    ip, dp, vp = p.split()[0], p.split()[1], p.split()[2]

    image = cv2.imread(ip)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_cor = image
    image_cor = image_cor_process(image_cor)
    
    event_voxel = np.load(vp)
    event_voxel = event_voxel.transpose(1, 2, 0)
    event_voxel = voxel_cor_process(event_voxel)
    event_voxel = event_voxel.transpose(2, 0, 1)

In [ ]:
import cv2
import os
# from dataset.transform import Image_Corruption
import random
def mask_left(image):
    height, width = image.shape[:2]

    left_half = width // 2

    image[:, :left_half] = 0
    return image

def mask_right(image):
    height, width = image.shape[:2]

    left_half = width // 2

    image[:, left_half:] = 0
    return image

img_cor = Image_Corruption(mask_pro=1, mask_radio=0.3, brightness_range=(0.5, 1.5), cor_types=["blur"])

In [ ]:
def corimg(img_path, img_corer):
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = img_corer(image)
    return image

In [24]:
img_cor = Image_Corruption(mask_pro=1, num_masks=1, mask_radio=1, cor_types=["overexpose"], exposure_beta=100, exposure_alpha=2)
# img_cor = Image_Corruption(mask_pro=1, num_masks=2, mask_radio=0.3, cor_types=["mask"])

In [25]:
in_dir = "/data_nvme/sph/DENSE/test/seq0/rgb/frames"
out_dir = "/data_nvme/sph/DENSE/test/seq0/rgb/frames_exp100"
os.makedirs(out_dir, exist_ok=True)

imgs = os.listdir(in_dir)

In [26]:
for p in imgs:
    if p.endswith("txt"):
        continue
    pa = os.path.join(in_dir, p)
    
    image = cv2.imread(pa)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # image = mask_left(image)
    image = img_cor(image)
    
    out_p = os.path.join(out_dir, p)
    cv2.imwrite(out_p, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

In [ ]:
path_list = [
    "/home/sph/event/da2-prompt-tuning/dataset/splits/mvsec/outdoor_day1.txt",
    "/home/sph/event/da2-prompt-tuning/dataset/splits/mvsec/outdoor_day2.txt",
    "/home/sph/event/da2-prompt-tuning/dataset/splits/mvsec/outdoor_night1.txt",
    "/home/sph/event/da2-prompt-tuning/dataset/splits/mvsec/outdoor_night2.txt",
    "/home/sph/event/da2-prompt-tuning/dataset/splits/mvsec/outdoor_night3.txt"
]

In [ ]:
# total = []
# for sp in path_list:
#     with open(sp, 'r') as f:
#         lines = f.readlines()
#         total.extend(lines)

# sel = random.sample(total, 1000)

with open("/home/sph/event/da2-prompt-tuning/da2_metric_depth/dataset/splits/mvsec/sel.txt", 'r') as f:
    sel = f.readlines()

In [ ]:
dir_ = "/home/sph/event/da2-prompt-tuning/dataset/splits/mvsec"
# sel_splits = os.path.join(dir_, "sel.txt")
# with open(sel_splits, 'w') as f:
#     f.writelines(sel)

out_dir = "/data_nvme/sph/mvsec_processed/img_blur"
os.makedirs(out_dir, exist_ok=True)

new_lines = []
for line in sel:
    img_p = line.split()[0]
    dep_p = line.split()[1]
    vox_p = line.split()[2].strip()

    cored_img = corimg(img_p, img_cor)
    
    scene = img_p.split("/")[-3]
    name = img_p.split("/")[-1]
    out_path = os.path.join(out_dir, f"{scene}_{name}")
    
    cv2.imwrite(out_path, cv2.cvtColor(cored_img, cv2.COLOR_RGB2BGR))
    new_lines.append(f"{out_path} {dep_p} {vox_p}\n")

cor_split = os.path.join(dir_, "blur.txt")
with open(cor_split, 'w') as f:
    f.writelines(new_lines)

In [ ]:
print(len(sel))
print(cor_split)

In [ ]:
import cv2
import os
# from dataset.transform import Image_Corruption
import random
import numpy as np

class Image_Corruption(object):
    def __init__(
        self,
        num_masks=3,
        mask_radio=0.2,
    ):

        self.num_masks = num_masks
        self.mask_radio = mask_radio

    def generate_masks(self, image_shape, num_masks, width_ratio=0.2, height_ratio=0.2):
        assert width_ratio <= 1
        assert height_ratio <= 1

        height, width, _ = image_shape
        masks = []

        # Convert ratios to actual dimensions
        w = int(width * width_ratio)
        h = int(height * height_ratio)

        for _ in range(num_masks):
            mask = np.zeros((height, width), dtype=np.uint8)
            # Generate a rectangle mask with size based on the percentage of the image dimensions
            x = random.randint(0, width - w) if width > w else 0
            y = random.randint(0, height - h) if height > h else 0
            cv2.rectangle(mask, (x, y), (x + w, y + h), 255, -1)

            masks.append(mask)

        return masks

    def mask_region(self, image, mask, mask_color=(0, 0, 0)):
        colored_mask = np.zeros_like(image)
        colored_mask[:] = mask_color
        return np.where(mask[..., None] == 255, colored_mask, image)
    
    def __call__(self, image):
        
        masks = self.generate_masks(
            image_shape=image.shape,
            num_masks=self.num_masks,
            width_ratio=self.mask_radio,
            height_ratio=self.mask_radio,
        )
        for mask in masks:
            image = self.mask_region(image, mask, mask_color=(0, 0, 0))

        return image, masks
    
img_cor = Image_Corruption(mask_radio=0.3)

In [ ]:
in_dir = "/data_nvme/sph/DENSE/test/seq0/events/voxels"
eve_frame = "/data_nvme/sph/DENSE/test/seq0/events/frames_white"
out_dir = "/data_nvme/sph/DENSE/test/seq0/events/voxels_cor"
os.makedirs(out_dir, exist_ok=True)

voxs = os.listdir(in_dir)
for p in imgs:
    if p.endswith("txt"):
        continue
    pa = os.path.join(in_dir, p)
    
    event_voxel = np.load(pa)
    event_voxel = event_voxel.transpose(1, 2, 0)
    
    event_voxel, masks = img_cor(event_voxel)
    event_voxel = event_voxel.transpose(2, 0, 1)
    
    out_p = os.path.join(out_dir, p)
    np.save(out_p, event_voxel)

In [ ]:
p = "/data_nvme/sph/DENSE/test/seq0/events/data/events_0000000000.npy"
e = np.load(p)
print(e.shape, e.dtype)
print(e[1])